In [1]:
import pandas as pd#importing all the necessary libraries
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import  roc_auc_score
from sklearn.multiclass import OneVsRestClassifier

In [2]:
x_=pd.read_csv('training_set_features.csv')#read all the training and testing csv files using pandas
y_=pd.read_csv('training_set_labels.csv')
x__=pd.read_csv('test_set_features.csv')

In [3]:
x_.shape

(26707, 36)

In [4]:
x_=x_.loc[:,'respondent_id':'age_group']#removing all the necessary features that will not of any use in predicting the target
x__=x__.loc[:,'respondent_id':'age_group']
x_.drop('health_insurance',inplace=True,axis=1)
x__.drop('health_insurance',inplace=True,axis=1)

***PREPROCESSING***

In [5]:
from sklearn.preprocessing import OneHotEncoder

In [6]:
one_hot_encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')

In [7]:
categorical_features = ['age_group']

In [8]:
encoded_features = one_hot_encoder.fit_transform(x_[categorical_features])

C:\Users\ASUS\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [9]:
encoded_feature_names = one_hot_encoder.get_feature_names_out(categorical_features)

In [10]:
x_encoded = pd.DataFrame(encoded_features, columns=encoded_feature_names)
x__encoded = pd.DataFrame(encoded_features, columns=encoded_feature_names)

In [11]:
x = pd.concat([x_.drop(columns=categorical_features),x_encoded], axis=1)
x1 = pd.concat([x_.drop(columns=categorical_features),x_encoded], axis=1)

In [12]:
x

,respondent_id,xyz_concern,xyz_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,opinion_xyz_risk,opinion_xyz_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,age_group_18 - 34 Years,age_group_35 - 44 Years,age_group_45 - 54 Years,age_group_55 - 64 Years,age_group_65+ Years
0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,1.0,2.0,2.0,1.0,2.0,0.0,0.0,0.0,1.0,0.0
1,1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,4.0,4.0,4.0,2.0,4.0,0.0,1.0,0.0,0.0,0.0
2,2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,4.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0
3,3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,3.0,5.0,5.0,4.0,1.0,0.0,0.0,0.0,0.0,1.0
4,4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,3.0,2.0,3.0,1.0,4.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26702,26702,2.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,5.0,2.0,2.0,0.0,0.0,0.0,0.0,1.0
26703,26703,1.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,2.0,2.0,5.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
26704,26704,2.0,2.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,...,4.0,2.0,5.0,4.0,2.0,0.0,0.0,0.0,1.0,0.0
26705,26705,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,1.0,2.0,2.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0


In [13]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [14]:
imputer = IterativeImputer(random_state=0)

In [15]:
x_train= pd.DataFrame(imputer.fit_transform(x), columns=x.columns)
x_test= pd.DataFrame(imputer.fit_transform(x1), columns=x1.columns)

In [16]:
x_test.isnull().sum()

respondent_id                  0
xyz_concern                    0
xyz_knowledge                  0
behavioral_antiviral_meds      0
behavioral_avoidance           0
behavioral_face_mask           0
behavioral_wash_hands          0
behavioral_large_gatherings    0
behavioral_outside_home        0
behavioral_touch_face          0
doctor_recc_xyz                0
doctor_recc_seasonal           0
chronic_med_condition          0
child_under_6_months           0
health_worker                  0
opinion_xyz_vacc_effective     0
opinion_xyz_risk               0
opinion_xyz_sick_from_vacc     0
opinion_seas_vacc_effective    0
opinion_seas_risk              0
opinion_seas_sick_from_vacc    0
age_group_18 - 34 Years        0
age_group_35 - 44 Years        0
age_group_45 - 54 Years        0
age_group_55 - 64 Years        0
age_group_65+ Years            0
dtype: int64

***LOGISTIC REGRESSION MODEL***

In [17]:
from sklearn.linear_model import LogisticRegression

In [18]:
model=LogisticRegression(solver='newton-cholesky',max_iter=10000)

In [19]:
from sklearn.model_selection import cross_val_score

In [20]:
y1=y_['xyz_vaccine']

***CALCULATING THE CROSS VALDATION SCORE TO CHECK THE ACCURACY OF THE MODEL***

In [21]:
cross_val_score(model,x_train,y1,cv=7,scoring='accuracy').mean()

0.8334520306304969

***TRAINING THE LOGISTIC REGRESSION MODEL WITH THE TRAINING DATASET***

In [22]:
model.fit(x_train, y1)

LogisticRegression(max_iter=10000, solver='newton-cholesky')

***PREDICTING THE PROBABLITIES OF THE TARGET VARIABLE***

In [23]:
y_pred_proba = model.predict_proba(x_test)[:, 1]

In [24]:
y_pred_proba

array([0.02810329, 0.20385262, 0.03858753, ..., 0.28633652, 0.02303637,
       0.15513294])

***CREATING THE OUTPUT DATASET***

In [25]:
y_test1=x_train['respondent_id']

In [26]:
y_test= pd.DataFrame(y_test1, columns=['respondent_id'])

In [27]:
y_test

,respondent_id
0,0.0
1,1.0
2,2.0
3,3.0
4,4.0
...,...
26702,26702.0
26703,26703.0
26704,26704.0
26705,26705.0


In [28]:
column_data = y_pred_proba.reshape(-1, 1)

In [29]:
column_data

array([[0.02810329],
       [0.20385262],
       [0.03858753],
       ...,
       [0.28633652],
       [0.02303637],
       [0.15513294]])

In [30]:
df = pd.DataFrame(column_data, columns=['xyz_vaccine'])

In [31]:
df

,xyz_vaccine
0,0.028103
1,0.203853
2,0.038588
3,0.084691
4,0.033180
...,...
26702,0.040257
26703,0.498994
26704,0.286337
26705,0.023036


In [32]:
y_test = pd.concat([y_test, df], axis=1)

In [33]:
y_test

,respondent_id,xyz_vaccine
0,0.0,0.028103
1,1.0,0.203853
2,2.0,0.038588
3,3.0,0.084691
4,4.0,0.033180
...,...,...
26702,26702.0,0.040257
26703,26703.0,0.498994
26704,26704.0,0.286337
26705,26705.0,0.023036


***ALL THE SIMILAR STEPS FOR seasonal_vaccine ALSO***

In [34]:
y2=y_['seasonal_vaccine']

In [35]:
cross_val_score(model,x_train,y2,cv=5,scoring='accuracy').mean()

0.7719325455804791

In [36]:
model.fit(x_train, y2)

LogisticRegression(max_iter=10000, solver='newton-cholesky')

In [37]:
y_pred_proba1 = model.predict_proba(x_test)[:, 1]

In [38]:
column_data1 = y_pred_proba1.reshape(-1, 1)
column_data1

array([[0.04517101],
       [0.15213997],
       [0.10905584],
       ...,
       [0.71131904],
       [0.02248354],
       [0.39869599]])

In [39]:
df1 = pd.DataFrame(column_data1, columns=['seasonal_vaccine'])
df1

,seasonal_vaccine
0,0.045171
1,0.152140
2,0.109056
3,0.946213
4,0.056131
...,...
26702,0.428014
26703,0.615163
26704,0.711319
26705,0.022484


In [40]:
y_test = pd.concat([y_test, df1], axis=1)

In [41]:
y_test

,respondent_id,xyz_vaccine,seasonal_vaccine
0,0.0,0.028103,0.045171
1,1.0,0.203853,0.152140
2,2.0,0.038588,0.109056
3,3.0,0.084691,0.946213
4,4.0,0.033180,0.056131
...,...,...,...
26702,26702.0,0.040257,0.428014
26703,26703.0,0.498994,0.615163
26704,26704.0,0.286337,0.711319
26705,26705.0,0.023036,0.022484


***CHANGING THE DATA TYPE OF THE REGISTRATION COLUMN FROM FLOAT TO INT***

In [42]:
y_test['respondent_id'] = y_test['respondent_id'].astype(int)

***SAVING THE OUTPUT .csv FILE***

In [43]:
csv_file_path = r'C:\Users\ASUS\Desktop\results.csv'
y_test.to_csv(csv_file_path, index=False,encoding='utf-8')